# Quick Start

## 1. Install Dependencies

First, we need to install the `lyric-py` package.

In [6]:
!pip install "lyric-py>=0.1.3"

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 7.2 MB/s eta 0:00:00MB/s eta 0:00:01:01


## 2. Install Language Worker

Next, we need to install Python webassembly worker and JavaScript webassembly worker.

In [7]:
!pip install "lyric-py-worker>=0.1.3" "lyric-js-worker>=0.1.3"

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 4.0 MB/s eta 0:00:00 MB/s eta 0:00:0102
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 6.4 MB/s eta 0:00:006.7 MB/s eta 0:00:01


## 3. Start Lyric Driver

To submit your code to the lyric workers, you need to start the lyric driver.

In [1]:
from lyric import DefaultLyricDriver

lcd = DefaultLyricDriver(host="127.0.0.1", log_level="ERROR")
lcd.start()

# Load workers(default: Python, JavaScript)
await lcd.lyric.load_default_workers()

## 3.1 Execute Python Code

You can submit a Python code by `exec` method. It will be executed by the Python webassembly worker, 
then you can get the execution status and result.

In [2]:
python_code = """
print("Hello, World!, I am a Python program.")

def add(a, b):
    return a + b

result = add(1, 2)
print(result)
"""

py_res = await lcd.exec(python_code, "python")
print(py_res)

["local_778429183bca2759@782d7f19-d4ee-433f-bfa3-b3f2d3db2ee0"]: "[Python-InterpreterTask] script: InterpreterRequest(resources=None, protocol=1, lang='PYTHON', code='\\nprint(\"Hello, World!, I am a Python program.\")\\n\\ndef add(a, b):\\n    return a + b\\n\\nresult = add(1, 2)\\nprint(result)\\n')"
["local_778429183bca2759@782d7f19-d4ee-433f-bfa3-b3f2d3db2ee0"]: "Hello, World!, I am a Python program."
["local_778429183bca2759@782d7f19-d4ee-433f-bfa3-b3f2d3db2ee0"]: "3"
["local_778429183bca2759@782d7f19-d4ee-433f-bfa3-b3f2d3db2ee0"]: "[Python-InterpreterTask] test_result: b'{\"content\":\"Execute script successfully\",\"exit_code\":0,\"lang\":\"Python\",\"protocol\":1,\"stderr\":\"\",\"stdout\":\"Hello, World!, I am a Python program.\\\\n3\\\\n\",\"success\":true}'"
CodeResult(exit_code=0, stdout='Hello, World!, I am a Python program.\n3\n', stderr='', output=None)


In [3]:
# Let's show the output of the Python code
py_res

CodeResult(exit_code=0, stdout='Hello, World!, I am a Python program.\n3\n', stderr='', output=None)

As you can see, the Python code is executed successfully, and the result like below.

```python
CodeResult(exit_code=0, stdout='Hello, World!, I am a Python program.\n3\n', stderr='', output=None)
```

### 3.2 Execute JavaScript Code

You can submit a JavaScript code by `exec` method with `lang="javascript"` parameter. It will be executed by the JavaScript webassembly worker, then you can get the execution status and result.

In [4]:
js_code = """
console.log("Hello, World!, I am a JavaScript program.");

function add(a, b) {
    return a + b;
}

let result = add(1, 2);
console.log(result);
"""

js_res = await lcd.exec(js_code, "javascript")
print(js_res)

["local_3a19f559b51bdd2c@4e16b2bf-e167-4b37-80cc-a50d31800b38"]: "[JavaScript-InterpreterTask] script: {\"protocol\":1,\"lang\":\"JAVASCRIPT\",\"code\":\"\\nconsole.log(\\\"Hello, World!, I am a JavaScript program.\\\");\\n\\nfunction add(a, b) {\\n    return a + b;\\n}\\n\\nlet result = add(1, 2);\\nconsole.log(result);\\n\"}"
["local_3a19f559b51bdd2c@4e16b2bf-e167-4b37-80cc-a50d31800b38"]: "Begin to run javascript code in interpreter Task"
["local_3a19f559b51bdd2c@4e16b2bf-e167-4b37-80cc-a50d31800b38"]: "Hello, World!, I am a JavaScript program."
["local_3a19f559b51bdd2c@4e16b2bf-e167-4b37-80cc-a50d31800b38"]: "3"
CodeResult(exit_code=0, stdout='Begin to run javascript code in interpreter Task\nHello, World!, I am a JavaScript program.\n3', stderr='', output=None)


In [5]:
# Let's show the output of the JavaScript code
js_res

CodeResult(exit_code=0, stdout='Begin to run javascript code in interpreter Task\nHello, World!, I am a JavaScript program.\n3', stderr='', output=None)

As you can see, the JavaScript code is executed successfully, and the result like below.

```python
CodeResult(exit_code=0, stdout='Before run code in interpreter Task\nHello, World!, I am a JavaScript program.\n3\nAfter run code in interpreter Task', stderr='', output=None)
```

### 3.3 Execute Python Function

You can submit a Python function by `exec1` method. It will be executed by the Python webassembly worker, then you can get the execution status and result.

Note: **You must pass the function name to be executed as a parameter.**

In [6]:
import json

py_func = """
# Define a function that take a dictionary as input and return a dictionary

def message_handler(message_dict):
    user_message = message_dict.get("user_message")
    ai_message = message_dict.get("ai_message")
    return {
        "user": user_message,
        "ai": ai_message,
        "all": [user_message, ai_message],
        "custom": "custom",
        "handler_language": "python",
    }
"""

# Create a input dictionary
input_data = {
    "user_message": "Hello from user",
    "ai_message": "Hello from AI",
}
# Convert the input dictionary to json bytes, as the input to the function(It will be converted to dictionary inside the function)
input_bytes = json.dumps(input_data).encode("utf-8")

py_func_res = await lcd.exec1(py_func, input_bytes, "message_handler", lang="python")
print("Python result:", py_func_res.output)
print("Full output:", py_func_res)

["local_778429183bca2759@782d7f19-d4ee-433f-bfa3-b3f2d3db2ee0"]: "[Python-InterpreterTask] script: InterpreterRequest(resources=None, protocol=1, lang='PYTHON', code='\\n# Define a function that take a dictionary as input and return a dictionary\\n\\ndef message_handler(message_dict):\\n    user_message = message_dict.get(\"user_message\")\\n    ai_message = message_dict.get(\"ai_message\")\\n    return {\\n        \"user\": user_message,\\n        \"ai\": ai_message,\\n        \"all\": [user_message, ai_message],\\n        \"custom\": \"custom\",\\n        \"handler_language\": \"python\",\\n    }\\n')"
["local_778429183bca2759@782d7f19-d4ee-433f-bfa3-b3f2d3db2ee0"]: "[Python-InterpreterTask] call_name: message_handler"
Python result: {'ai': 'Hello from AI', 'all': ['Hello from user', 'Hello from AI'], 'custom': 'custom', 'handler_language': 'python', 'user': 'Hello from user'}
Full output: CodeResult(exit_code=0, stdout='', stderr='', output={'ai': 'Hello from AI', 'all': ['Hello fro

In [7]:
# Let's show the output of the Python function(It will be a dictionary)
py_func_res.output

{'ai': 'Hello from AI',
 'all': ['Hello from user', 'Hello from AI'],
 'custom': 'custom',
 'handler_language': 'python',
 'user': 'Hello from user'}

### 3.4 Execute JavaScript Function

You can submit a JavaScript function by `exec1` method with `lang="javascript"` parameter. It will be executed by the JavaScript webassembly worker, then you can get the execution status and result.

In [8]:
js_func = """
// Define a function that take a dictionary as input and return a dictionary

function message_handler(message_dict) {
    return {
        user: message_dict.user_message,
        ai: message_dict.ai_message,
        all: [message_dict.user_message, message_dict.ai_message],
        custom: "custom",
        handler_language: "javascript",
    };
}
"""

js_func_res = await lcd.exec1(js_func, input_bytes, "message_handler", lang="javascript")
print("JavaScript result:", js_func_res.output)
print("Full output:", js_func_res)

["local_3a19f559b51bdd2c@4e16b2bf-e167-4b37-80cc-a50d31800b38"]: "[JavaScript-InterpreterTask] script: {\"protocol\":1,\"lang\":\"JAVASCRIPT\",\"code\":\"\\n// Define a function that take a dictionary as input and return a dictionary\\n\\nfunction message_handler(message_dict) {\\n    return {\\n        user: message_dict.user_message,\\n        ai: message_dict.ai_message,\\n        all: [message_dict.user_message, message_dict.ai_message],\\n        custom: \\\"custom\\\",\\n        handler_language: \\\"javascript\\\",\\n    };\\n}\\n\"}"
["local_3a19f559b51bdd2c@4e16b2bf-e167-4b37-80cc-a50d31800b38"]: "[JavaScript-InterpreterTask] call_name: message_handler"
["local_3a19f559b51bdd2c@4e16b2bf-e167-4b37-80cc-a50d31800b38"]: "The code is: "
["local_3a19f559b51bdd2c@4e16b2bf-e167-4b37-80cc-a50d31800b38"]: ""
["local_3a19f559b51bdd2c@4e16b2bf-e167-4b37-80cc-a50d31800b38"]: "// Define a function that take a dictionary as input and return a dictionary"
["local_3a19f559b51bdd2c@4e16b2bf-e1

In [9]:
# Let's show the output of the JavaScript function(It will be a dictionary)
js_func_res.output

{'ai': 'Hello from AI',
 'all': ['Hello from user', 'Hello from AI'],
 'custom': 'custom',
 'handler_language': 'javascript',
 'user': 'Hello from user'}

## 4. Stop Lyric Driver

Finally, you can stop the lyric driver by calling the `stop` method.

In [10]:
lcd.stop()